# Capítulo 9 - Uso de Django para crear y desplegar apps web
___

## Crear un proyecto Django
___
Para crear una pagina web instalamos Django:

``` shell
pip install django whitenoise
```

Creamos un nuevo repositorio en Github y la llamamos `webproject`. Clonamos y abrimos una consola en la raíz con estos comandos:

``` shell
cd ~/workspaces
git clone git@github.com:{user_github}/webproject.git
cd webproject
django-admin startproject webproject .
```

Ahí vemos que hemos creado esta carpetas y archivos

```
workspaces/
  +- webproject/  (raíz del repositorio)
      +- webproject/
      |   + settings.py
      |  ...
      +-manage.py
```

Añadimos un archivo nuevo en la raíz `requirements.txt` con el contenido:

```
django
whitenoise
```

Ahora haremos unos cambios en `webproject/settings.py`:

```python
ALLOWED_HOSTS = ['localhost', '127.0.0.1', '.pythonanywhere.com']

MIDDLEWARE = [
    # 'django.middleware.security.SecurityMiddleware',  # esta línea se comenta
    'whitenoise.middleware.WhiteNoiseMiddleware',       # esta línea se añade
    ...                                                 # el resto igual
]
    
LANGUAGE_CODE = 'es-es'

TIME_ZONE = 'Europe/Madrid'

STATIC_URL = '/static/'
STATIC_ROOT = 'static_root'
STATICFILES_DIRS = ['static']
MEDIA_URL = '/media/'
MEDIA_ROOT = 'media_root'
```

ejecutamos estos comando para regenerar la base de datos y reunir los archivos estáticos:

```shell
cd ~/workspaces/webproject
python manage.py migrate
python manage.py collectstatic
```

para que los archivo autogenerados no se suban al repositorio añadimos un `.gitignore` con:

```
static_root
db.sqlite3
```

levantamos el servidor con los comandos:

```shell
python manage.py runserver 80
```

Sin cerrar la consola vamos a url http://localhost y seberíamos ver nuestro server levantado

## Crear una cuenta en pythonanywhere
___
1. visitamos https://www.pythonanywhere.com/
2. le damos a crear una cuenta (singup), ponemos un nombre de usuario y email (el dominio finalmente será https://{user}.pythonanywhere.com)
3. vamos arriba a la derecha "Cuenta" > "API token", pulsamos en generar token

## Despliege
___
vamos a "Dashboard" > "New console" > "Bash" y ponemos los siguientes comandos: 

```shell
pip3.9 install --user pythonanywhere
pa_autoconfigure_django.py --nuke --python=3.9 https://github.com/{user_github}/webproject
```
Nota: `--nuke` hace que destruya totalmente otro proyecto anteriormente desplegado en la cuenta actual de pythonanywhere

Esperamos a que se ejecuten todos los procesos en el servidor y finalmente la web estaría abierta al público en la url: https://{user}.pythonanywhere.com (sustituyendo "user"). Como el proceso se levanta en segundo plano, opcinalmente podríamos finalizar esta consola bash desde el "menú" > "Consolas".

Cada vez que creemos nuevo código y lo subamos a Github, para a actualizar nuestra web pública, volvemos a crear una consola bash y ejecutamos estos comandos (con el user correspondiente):

```shell
cd {user_pythonanywhere}.pythonanywhere.com
git pull
```

Django automaticamente detecta cambios y relanza la web, pero cierto cambios, como pueden ser las URLs requieren un reinicio total. Para reiniciar Debemos ir a "Web" y pulsar en "Reload {user_pythonanywhere}.pythonanywhere.com"

Por último, ahí mismo, en "Web", podemos ver que para mantener el sitio abierto hay que pulsar "Run until 3 month from today" cada cierto tiempo, ya que estamos usando una cuenta no de pago

## Añadir funcionalidad
___
habrimos un terminal en la raíz del repositorio e introducimos este comando, que creará una app llamada `testapp`:

``` shell
python manage.py startapp testapp
```

Ahora haremos unos cambios en `webproject/settings.py`:

``` python
INSTALLED_APPS = [
    'testapp',      # añadir esta línea
    ...
]
```


### Servicio REST tipo GET

En el archivo `testapp/views.py` incluimos el siguiente método, cuya respuesta será un JSON que calculamos con los datos de la llamada:

``` python
from datetime import datetime
from django.http import JsonResponse

def get_example(request):
    response = {'request': {'time': datetime.now().isoformat(),
                            'method': request.method,
                            'path': request.path,
                            'params': request.GET,
                            'headers': dict(request.headers)}}
    return JsonResponse(response, safe=False, json_dumps_params={'indent': 2})
```

Abrimos el archivo `webproject/urls.py` he incluimos una nueva ruta:

``` python
import testapp.views

urlpatterns = [
    ...
    path('get_example', testapp.views.get_example),
]
```

Si iniciamos el servidor ya tendríamos esta respuesta en http://127.0.0.1/get_example

### Servicio REST tipo POST

En el archivo `testapp/views.py` incluimos el siguiente método, cuya respuesta será un JSON que calculamos con los datos de la llamada:

``` python
from datetime import datetime
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def post_example(request):
    response = {'request': {'time': datetime.now().isoformat(),
                'method': request.method,
                'path': request.path,
                'params': request.GET,
                'headers': dict(request.headers),
                'body': request.body.decode()}}
    return JsonResponse(response, safe=False, json_dumps_params={'indent': 2})
```

Abrimos el archivo `urls.py` he incluimos una nueva ruta:

``` python
import testapp.views

urlpatterns = [
    ...
    path('post_example', testapp.views.post_example),
]
``` 

Si iniciamos el servidor ya tendríamos esta respuesta en http://127.0.0.1/post_example

### Página web sin parámetros URL

En el archivo `testapp/views.py` incluimos el siguiente método, cuya respuesta será un html que renderizamos con los datos de la llamada:

``` python
from django.shortcuts import render

def gallery(request):
    return render(request, 'testapp/gallery.html')
```

también creamos este archivo `.html` en la siguiente ruta de carpetas: `testapp/templates/testapp/gallery.html`

``` html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Gallery</title>
    <link rel="icon" href="/static/favicon.png" sizes="32x32" type="image/png">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous"></script>
</head>
<body style="background-color:black;">
<nav class="navbar navbar-dark bg-success">
    <div class="container-fluid"><a class="navbar-brand" href="/testapp/gallery">Gallery</a></div>
</nav>
<br/>
<div class="container">
    <div class="list-group">
        <a href="/gallery/photo1" class="list-group-item list-group-item-action">photo 1</a>
        <a href="/gallery/photo2" class="list-group-item list-group-item-action">photo 2</a>
    </div>
</div>
</body>
</html>
```

y en la ruta `/static/` un archivo que haremos con paint y llamaremos `favicon.png` el cual será el icono de la web 

Abrimos el archivo `urls.py` he incluimos una nueva ruta:

``` python
import testapp.views

urlpatterns = [
    ...
    path('gallery', testapp.views.gallery),
]
``` 

Si iniciamos el servidor ya tendríamos esta respuesta en http://127.0.0.1/gallery se trata de una lista con dos links "http://127.0.0.1/gallery/photo1" y "http://127.0.0.1/gallery/photo2"

### Página web con parámetros url

En el archivo `testapp/views.py` incluimos el siguiente método, cuya respuesta será un html que renderizamos con los datos de la llamada:

``` python
from django.shortcuts import render

def gallery_photo(request, photo):
    context = {'photo': photo}
    return render(request, 'testapp/gallery_photo.html', context)
```

el context es un diccionario con los valores que tomarán las variables entre {{ }} de la respuesta, en este caso {{ photo }}

también creamos este archivo `.html` en la siguiente ruta de carpetas: `testapp/templates/testapp/gallery_photo.html`

``` html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Gallery</title>
    <link rel="icon" href="/static/favicon.png" sizes="32x32" type="image/png">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous"></script>
</head>
<body style="background-color:black;">
<nav class="navbar navbar-dark bg-success">
    <div class="container-fluid"><a class="navbar-brand" href="/testapp/gallery">Gallery</a></div>
</nav>
<br/>
<div class="container">
    <div class="list-group"><img src="/static/{{ photo }}.png" class="img-thumbnail"></div>
</div>
</body>
</html>
```

y en la ruta `/static/` dos archivos que haremos con paint y llamaremos `photo1.png` y `photo2.png` los cuales serán las fotos mostradas

Abrimos el archivo `urls.py` he incluimos una nueva ruta, donde `<photo>` en la URL quiere decir que será el valor de la variable método de la respuesta:

``` python
import testapp.views

urlpatterns = [
    ...
    path('gallery/<photo>', testapp.views.gallery_photo),
]
``` 

Si iniciamos el servidor ya tendríamos esta respuesta en "http://127.0.0.1/gallery/photo1" y "http://127.0.0.1/gallery/photo2"

## Securizar la secret-key de django
___
Hasta ahora no hemos tenido en cuenta que estamos exponiendo la clave secreta de Django, seguramente Github ya no haya avisado de este agujero de seguridad por email. Para resolver esto haremos que Django lea la clave de un archivo previamente subido al sistema de archivos de pythonanywhere, sin compartirlo con nadie más ni sin subirlo a Github. Para ello primero añadimos una línea con `.secret_key` al final del archivo `.gitignore`:

```
static_root
db.sqlite3
.secret_key
```

luego creamos el archivo `.secret_key` en la raíz del proyecto, y escribimos algo aleatorio pero no igual a lo que se encuentra actualmente en `webproject/settings.py`, por ejemplo: 

```
django-insecure-123456asdfASDF?:%&
```
Nota: no copiar esta literamente tampoco

Ahora modificamos `webproject/settings.py` para que lea y cargue este archivo como clave secreta:

```python
SECRET_KEY = open('.secret_key', 'r', encoding='utf-8').read()
```

También subimos el archivo `.secret_key` a pythonanywhere yendo a "Files" y pulsando en "Upload a file" y lo seleccionamos

Por último, pusheamos los cambios a Github y actualizamos el proyecto desde una consola de pythonanywhere

## Ir más allá
___
A partir de aquí se puede seguir un tutorial de Django para añadir alguna funcionalidad, como pueda ser cuestionarios ("forms"), por ejemplo: https://docs.djangoproject.com/en/3.2/intro/tutorial01/#creating-the-polls-app

Para el tema de estilos se puede usar [Bootstrap 5](https://getbootstrap.com/docs/5.0/layout/grid/), en los ejemplos anteriores se puede ver como está importado en el `<head>`.